In [1]:
!pip install langchain transformers huggingface_hub requests

In [3]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [20]:
!pip install tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.2 MB/s eta 0:00:00


In [21]:
import os
import requests
from langchain.llms import HuggingFaceHub
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from google.colab import userdata
from tavily import TavilyClient

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [5]:
HUGGINGFACE_API_KEY = userdata.get("HF_TOKEN")

In [8]:
# llm = HuggingFaceHub(
#     repo_id="google/flan-t5-large",
#     model_kwargs={"temperature": 0.7, "max_length": 512},
#     huggingfacehub_api_token=HUGGINGFACE_API_KEY
# )


In [13]:
# llm = HuggingFaceHub(
#     repo_id="bigscience/T0pp",  # Alternative model
#     model_kwargs={"temperature": 0.7, "max_length": 512},
#     huggingfacehub_api_token=HUGGINGFACE_API_KEY
# )


In [24]:
TAVILY_API_KEY = userdata.get("TAVILY_API_KEY")
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

In [27]:
# Implement the real Tavily search:
# def get_profile_url_tavily(name: str) -> str:
#     """Search for a LinkedIn profile URL using Tavily."""
#     query = f"{name} LinkedIn profile site:linkedin.com"
#     response = tavily_client.search(query, max_results=1)
#     if response["results"]:
#         return response["results"][0]["url"]
#     return f"No LinkedIn profile found for {name}"

In [26]:
def get_profile_url_tavily(query: str) -> str:
    """
    Mock function to simulate getting a LinkedIn profile URL.
    In a real implementation, you'd use an actual web search API.
    """
    return f"https://www.linkedin.com/in/{query.lower().replace(' ', '-')}-profile"

In [33]:
def lookup(name: str) -> str:
    from langchain_core.prompts import PromptTemplate
    from langchain_core.tools import Tool
    from langchain.agents import create_react_agent, AgentExecutor
    from langchain import hub

    class MockLLM:
        def __call__(self, prompt):
            url =  f"https://www.linkedin.com/in/{name.lower().replace(' ', '-')}-profile"
            return f"Thought: I need to find the LinkedIn profile URL for {name}.\nAction: Crawl Google 4 linkedin profile pages\nAction Input: {name}\nObservation: {url}\nFinal Answer: {url}"

        def bind(self, **kwargs):
            return self

    llm = MockLLM()

    template = """given the full name {name_of_person} I want you to get me a link to their LinkedIn profile page.You should only give me only a url"""
    prompt_template = PromptTemplate(template=template, input_variables=["name_of_person"])

    tools_for_agent = [
        Tool(
            name="Crawl Google 4 linkedin profile pages",
            func=get_profile_url_tavily,
            description="Useful for when you need to get the LinkedIn page url"
        )
    ]

    react_prompt = hub.pull("hwchase17/react")
    agent = create_react_agent(llm=llm, tools=tools_for_agent, prompt=react_prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True, handle_parsing_errors=True)

    result = agent_executor.invoke(
        input={"input": prompt_template.format(name_of_person=name)}
    )

    linkedin_profile_url = result["output"]
    return linkedin_profile_url

print(lookup("David"))



> Entering new AgentExecutor chain...
Parsing LLM output produced both a final answer and a parse-able action:: Thought: I need to find the LinkedIn profile URL for David.
Action: Crawl Google 4 linkedin profile pages
Action Input: David
Observation: https://www.linkedin.com/in/david-profile
Final Answer: https://www.linkedin.com/in/david-profile
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseParsing LLM output produced both a final answer and a parse-able action:: Thought: I need to find the LinkedIn profile URL for David.
Action: Crawl Google 4 linkedin profile pages
Action Input: David
Observation: https://www.linkedin.com/in/david-profile
Final Answer: https://www.linkedin.com/in/david-profile
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseParsing LLM output produced both a final answer and a parse-ab

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
# FOR real time tavily search

# def lookup(name: str) -> str:
#     """Find LinkedIn profile using a Hugging Face model and Tavily-like function."""

#     template = """Given the full name {name_of_person}, provide a LinkedIn profile URL."""
#     prompt_template = PromptTemplate(template=template, input_variables=["name_of_person"])

#     tools_for_agent = [
#         Tool(
#             name="Get LinkedIn Profile URL",
#             func=get_profile_url_tavily,
#             description="Gets the LinkedIn profile URL based on a person's name."
#         )
#     ]

#
#     react_prompt = hub.pull("hwchase17/react")

#     agent = create_react_agent(llm=generator, tools=tools_for_agent, prompt=react_prompt)
#     agent_executor = AgentExecutor(agent=agent, tools=tools_for_agent, verbose=True)

#     result = agent_executor.invoke(
#         input={"input": prompt_template.format(name_of_person=name)}
#     )

#     linkedin_profile_url = result["output"]
#     return linkedin_profile_url

In [ ]:
# print(lookup("David"))